In [3]:
import arcpy
arcpy.env.overwriteOutput = True 
from arcpy.sa import *
from arcpy import env

import os
import pandas as pd
import numpy as np


workspace = os.path.join("..", 'RESULTS')
#make sure workspace directory exists
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
tempspace = os.path.join(workspace, "temp")
#make sure workspace directory exists
if not os.path.exists(tempspace):
    os.makedirs(tempspace)


outspace = os.path.join(workspace, 'Outputs')
#make sure workspace directory exists
if not os.path.exists(outspace):
    os.makedirs(outspace)

    
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container { width: 95%;} div#menubar-container { width: 85%; } div#maintoolbar-container { width: 99%; } </style> """))

OK so gotta refactor to find a better way to extract all data from each scenario and compile into one frame. There will be
- Base Flooded 0m
- Base Flooded 1 m

- SLR 2ft flooded 0m reg recharge
- SLR 2ft flooded 1m reg recharge
- SLR 2ft flooded 0m RCP4.5 recharge
- SLR 2ft flooded 1m RCP4.5 recharge

- SLR 5ft flooded 0m reg recharge
- SLR 5ft flooded 1m reg recharge
- SLR 5ft flooded 0m RCP4.5 recharge
- SLR 5ft flooded 1m RCP4.5 recharge

- SLR 8ft flooded 0m reg recharge
- SLR 8ft flooded 1m reg recharge
- SLR 8ft flooded 0m RCP4.5 recharge
- SLR 8ft flooded 1m RCP4.5 recharge



SLR_coastline file from NOAA Processing Wrokflow
- simplify polygon on file 5 m  = Simplified
- run a union to remove the gaps, make sure gaps are not allowed manually edit to merge all polys together = gap_union
- run a union with the model square to get the land area as polygon  = square_union.shp
- at this point export square_uniin with no dumb fields make OceanCell fields 1 for land -1 fpr ocean
- union again on the 100 m buffer square for heads Model_square_100mBuf.shp  = buff_union  (Make a backup if mess up on editing)
- make ocean cells ibound field -1 and land 1
- Export as ibound_specHd_w_100m_buff_SLRXft

In [4]:
#ascs
DtW_base = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5p2', "dep2Wat__base.asc")
DtW_8ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR8ft', "dep2Wat_SLR8ft.asc")
DtW_5ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR5ft', "dep2Wat_SLR5ft.asc")
DtW_2ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR2ft', "dep2Wat_SLR2ft.asc")
DtW_1ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR1ft', "dep2Wat_SLR1ft.asc")

DtW_8ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR8ft', "dep2Wat_SLR8ft_RCP45.asc")
DtW_5ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR5ft', "dep2Wat_SLR5ft_RCP45.asc")
DtW_2ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR2ft', "dep2Wat_SLR2ft_RCP45.asc")

#  path to the infrastructure layers 
InfraShpPath = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped_clean')


Depths_to_clip_infrastructure = {"Surface": 0.01,  "Sub": 1.01}

SLR_scenarios = ["base", "2ft", "5ft", "8ft", "2ft_RCP45", "5ft_RCP45", "8ft_RCP45", "1ft",]    # Each is a raster in a separate folder

Scenaio_raster_dic = {
    "base":        DtW_base, 
    "2ft":         DtW_2ft,   
    "2ft_RCP45":   DtW_2ft_RCP45,    
    "5ft":         DtW_5ft,    
    "5ft_RCP45":   DtW_5ft_RCP45,    
    "8ft":         DtW_8ft,   
    "8ft_RCP45":   DtW_8ft_RCP45, 
    "1ft"      :   DtW_1ft}

Polyline_infrstructures = ['Sewer_Lines.shp', 'Water_Lines.shp', 'Roads.shp', 'Electrical_Lines.shp'] 

Point_infrstructures = ['ASPA_Customer_Meters.shp', 'Water_Boosters.shp', 'Sewer_Lift_Stations.shp', 'Buildings.shp'] 

In [5]:
# Create shapefiles and a dic of the total lengths of polyline files 

OG_lenth_dic = {}
for i in Polyline_infrstructures:
    
    shapefile_to_measure = os.path.join(InfraShpPath, i)
    
    # First copy the original features to calculate and print the total lengths of polylines later
    arcpy.management.CopyFeatures(shapefile_to_measure, os.path.join(tempspace, "Length_{}".format(i)))

    fc_OG = os.path.join(tempspace, "Length_{}".format(i))
    arcpy.AddField_management(fc_OG, 'lenOG_m','DOUBLE')
    arcpy.management.CalculateField(fc_OG, "lenOG_m", "!SHAPE.geodesicLength@METERS!")   
    field = arcpy.da.TableToNumPyArray (fc_OG, "lenOG_m", skip_nulls=True)
    tot_len = field["lenOG_m"].sum()
    print("Total Original Length of {} is {} m".format(i, tot_len)) 
    OG_lenth_dic[i] = tot_len
    
    
for i in Point_infrstructures:
    fc = os.path.join(InfraShpPath, i)
    field = arcpy.da.TableToNumPyArray (fc, "FID", skip_nulls=True)
    tot_things = len(field)
    OG_lenth_dic[i] = tot_things
    print("Total number of {} is {}".format(i, tot_things))
    

# Turn the Dic into a dataframe
OG_lenth = pd.DataFrame.from_dict([OG_lenth_dic], orient='columns').transpose()
OG_lenth = OG_lenth.rename(columns={0:"Original_length_or_number"})
OG_lenth

Total Original Length of Sewer_Lines.shp is 42315.5045899074 m
Total Original Length of Water_Lines.shp is 91195.43612632273 m
Total Original Length of Roads.shp is 76765.56681189116 m
Total Original Length of Electrical_Lines.shp is 20486.366790691205 m
Total number of ASPA_Customer_Meters.shp is 1329
Total number of Water_Boosters.shp is 5
Total number of Sewer_Lift_Stations.shp is 8
Total number of Buildings.shp is 2398


,Original_length_or_number
ASPA_Customer_Meters.shp,1329.000000
Buildings.shp,2398.000000
Electrical_Lines.shp,20486.366791
Roads.shp,76765.566812
Sewer_Lift_Stations.shp,8.000000
Sewer_Lines.shp,42315.504590
Water_Boosters.shp,5.000000
Water_Lines.shp,91195.436126


### Main cell: Assess flooded infrastructure

In [6]:
Flooded_scenarios_dic = {}

for depth in Depths_to_clip_infrastructure:
    
    for scenario in SLR_scenarios:
        
        # Create polygons for Surface or subsurface "flooded" areas for each of the scenarios in SLR_scenarios:
        in_raster = Scenaio_raster_dic[scenario]
        outraster = os.path.join(tempspace, 'Temp_raster.tif')
        DaNewraster = Raster(in_raster) < Depths_to_clip_infrastructure[depth]
        DaNewraster.save(outraster)

        # Convert the raster to polygons
        outPolygons = os.path.join(tempspace, 'temp.shp')
        arcpy.RasterToPolygon_conversion(outraster, outPolygons, "NO_SIMPLIFY")

        ## Delete any areas where the raster (now gridcode attribute) is 0
        arcpy.MakeFeatureLayer_management(outPolygons, "Temp_Poly")
        Selection = arcpy.SelectLayerByAttribute_management("Temp_Poly", "NEW_SELECTION", "gridcode > 0")
        arcpy.CopyFeatures_management(Selection, os.path.join(tempspace, 'Flooded_Area_{}_{}.shp'.format(scenario, depth)))
            
        results_dic = {}
        for file in os.listdir(InfraShpPath):
            if file.endswith(".shp"):
                shapefile_to_clip = os.path.join(InfraShpPath, file) 

                # Now do the actual clipping of flooded areas
                out_clipped  = os.path.join(outspace, "{}_{}_Flooded_{}".format(scenario, depth, file))
                arcpy.Clip_analysis(shapefile_to_clip, os.path.join(tempspace, 'Flooded_Area_{}_{}.shp'.format(scenario, depth)), out_clipped) 
                
                
                # NOW work on the Polyline_infrstructures:                                 
                if file in Polyline_infrstructures:
                    arcpy.AddField_management(out_clipped, 'lenFlod_m','DOUBLE')
                    arcpy.management.CalculateField(out_clipped, "lenFlod_m", "!SHAPE.geodesicLength@METERS!")   
                    field = arcpy.da.TableToNumPyArray (out_clipped, "lenFlod_m", skip_nulls=True)
                    flod_len = field["lenFlod_m"].sum()
                    print("Length of flooded {} is {} m in {} at {} depth".format(file, flod_len, scenario, depth)) 
                    
                    pct_flooded = round((flod_len/OG_lenth_dic[file])*100, 1)
                    print("Total length of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))
                   
                    results_dic[file] = flod_len
                    
                                      
               # NOW work on the point_infrstructures: 
                if file in Point_infrstructures:
                    field = arcpy.da.TableToNumPyArray (out_clipped, "FID", skip_nulls=True)
                    flood_things = len(field)
                    print("Number of flooded {} is {}".format(file, flood_things))
                    
                    pct_flooded = round((flood_things/OG_lenth_dic[file])*100, 1)
                    print("Total number of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))
                
                    results_dic[file] = flood_things     
        
        Flooded_scenarios_dic[scenario+"_"+depth] = results_dic
                

Number of flooded ASPA_Customer_Meters.shp is 1
Total number of ASPA_Customer_Meters.shp is 1329, Percent of flooded is 0.1%  

Number of flooded Buildings.shp is 1
Total number of Buildings.shp is 2398, Percent of flooded is 0.0%  

Length of flooded Electrical_Lines.shp is 402.05869717242 m in base at Surface depth
Total length of Electrical_Lines.shp is 20486.366790691205, Percent of flooded is 2.0%  

Length of flooded Roads.shp is 65.16813055348 m in base at Surface depth
Total length of Roads.shp is 76765.56681189116, Percent of flooded is 0.1%  

Number of flooded Sewer_Lift_Stations.shp is 0
Total number of Sewer_Lift_Stations.shp is 8, Percent of flooded is 0.0%  

Length of flooded Sewer_Lines.shp is 53.33366692783 m in base at Surface depth
Total length of Sewer_Lines.shp is 42315.5045899074, Percent of flooded is 0.1%  

Number of flooded Water_Boosters.shp is 0
Total number of Water_Boosters.shp is 5, Percent of flooded is 0.0%  

Length of flooded Water_Lines.shp is 220.0

Length of flooded Electrical_Lines.shp is 1481.887442832998 m in 1ft at Surface depth
Total length of Electrical_Lines.shp is 20486.366790691205, Percent of flooded is 7.2%  

Length of flooded Roads.shp is 1188.82290825052 m in 1ft at Surface depth
Total length of Roads.shp is 76765.56681189116, Percent of flooded is 1.5%  

Number of flooded Sewer_Lift_Stations.shp is 0
Total number of Sewer_Lift_Stations.shp is 8, Percent of flooded is 0.0%  

Length of flooded Sewer_Lines.shp is 585.2922128620671 m in 1ft at Surface depth
Total length of Sewer_Lines.shp is 42315.5045899074, Percent of flooded is 1.4%  

Number of flooded Water_Boosters.shp is 0
Total number of Water_Boosters.shp is 5, Percent of flooded is 0.0%  

Length of flooded Water_Lines.shp is 1345.327643873149 m in 1ft at Surface depth
Total length of Water_Lines.shp is 91195.43612632273, Percent of flooded is 1.5%  

Number of flooded ASPA_Customer_Meters.shp is 21
Total number of ASPA_Customer_Meters.shp is 1329, Percent 

Number of flooded Sewer_Lift_Stations.shp is 7
Total number of Sewer_Lift_Stations.shp is 8, Percent of flooded is 87.5%  

Length of flooded Sewer_Lines.shp is 13791.799940834026 m in 8ft_RCP45 at Sub depth
Total length of Sewer_Lines.shp is 42315.5045899074, Percent of flooded is 32.6%  

Number of flooded Water_Boosters.shp is 4
Total number of Water_Boosters.shp is 5, Percent of flooded is 80.0%  

Length of flooded Water_Lines.shp is 37698.79541113587 m in 8ft_RCP45 at Sub depth
Total length of Water_Lines.shp is 91195.43612632273, Percent of flooded is 41.3%  

Number of flooded ASPA_Customer_Meters.shp is 111
Total number of ASPA_Customer_Meters.shp is 1329, Percent of flooded is 8.4%  

Number of flooded Buildings.shp is 247
Total number of Buildings.shp is 2398, Percent of flooded is 10.3%  

Length of flooded Electrical_Lines.shp is 9525.97989035251 m in 1ft at Sub depth
Total length of Electrical_Lines.shp is 20486.366790691205, Percent of flooded is 46.5%  

Length of flood

In [14]:
# Post processing and syntehesis 

Flooded_scenarios_dic["Total_Lengths"] = OG_lenth_dic     # Add in the total lengths/numbers

output = pd.DataFrame.from_dict(Flooded_scenarios_dic, orient='columns')   # make it all a datframe
output = output.transpose()   # Transpose it

# Create a dataframe of percentage flooded
output_pct = output.copy()
for col in output: 
    tot_things = output[col].loc['Total_Lengths']
    output_pct[col] = output_pct[col]/tot_things
    
# Format percentages 
output_pct['sum']  = output_pct.sum(axis=1)/8
output_pct = output_pct.round(3)*100

# Sort by best to worst scenario 
output_pct = output_pct.sort_values("sum")

# Sort the in m df by best to worst scenario  
output['tot_Flooded_buried_lines'] = output[['Water_Lines.shp', 'Sewer_Lines.shp', "Electrical_Lines.shp"]].sum(axis=1)
output = output.sort_values('tot_Flooded_buried_lines')

#Save csvs
output_pct.to_csv(os.path.join("..", 'RESULTS', 'output_pct.csv'))
output.to_csv(os.path.join("..", 'RESULTS', 'output.csv'))

output_pct

,ASPA_Customer_Meters.shp,Buildings.shp,Electrical_Lines.shp,Roads.shp,Sewer_Lift_Stations.shp,Sewer_Lines.shp,Water_Boosters.shp,Water_Lines.shp,sum
base_Surface,0.1,0.0,2.0,0.1,0.0,0.1,0.0,0.2,0.3
2ft_Surface,0.1,0.0,3.8,0.4,0.0,0.3,0.0,0.6,0.6
2ft_RCP45_Surface,0.2,0.1,4.1,0.5,0.0,0.6,0.0,0.7,0.8
1ft_Surface,0.4,0.4,7.2,1.5,0.0,1.4,0.0,1.5,1.6
base_Sub,1.6,2.0,10.8,3.8,0.0,3.5,20.0,3.8,5.7
5ft_Surface,2.1,3.1,14.1,4.5,0.0,4.1,20.0,5.1,6.6
5ft_RCP45_Surface,2.1,3.2,14.2,4.7,0.0,4.2,20.0,5.2,6.7
2ft_Sub,4.9,6.5,28.7,13.0,0.0,11.2,20.0,11.5,12.0
2ft_RCP45_Sub,5.3,6.8,29.4,14.2,0.0,12.1,20.0,12.3,12.5
1ft_Sub,8.4,10.3,46.5,21.8,25.0,19.3,20.0,19.1,21.3


In [17]:
70145.876499/153997.307507


0.4555006683854619

In [15]:
output

,ASPA_Customer_Meters.shp,Buildings.shp,Electrical_Lines.shp,Roads.shp,Sewer_Lift_Stations.shp,Sewer_Lines.shp,Water_Boosters.shp,Water_Lines.shp,tot_Flooded_buried_lines
base_Surface,1.0,1.0,402.058697,65.168131,0.0,53.333667,0.0,220.026496,675.418860
2ft_Surface,1.0,1.0,777.364423,295.828089,0.0,134.207743,0.0,515.660873,1427.233040
2ft_RCP45_Surface,3.0,3.0,836.119598,417.045388,0.0,241.902982,0.0,627.133851,1705.156432
1ft_Surface,5.0,10.0,1481.887443,1188.822908,0.0,585.292213,0.0,1345.327644,3412.507300
base_Sub,21.0,49.0,2211.794692,2929.177709,0.0,1461.797169,1.0,3481.284504,7154.876365
5ft_Surface,28.0,74.0,2895.942563,3490.473860,0.0,1724.858733,1.0,4644.244144,9265.045440
5ft_RCP45_Surface,28.0,76.0,2900.994839,3582.715335,0.0,1789.481354,1.0,4764.074017,9454.550210
2ft_Sub,65.0,157.0,5872.152872,9955.054011,0.0,4724.822401,1.0,10483.346786,21080.322058
2ft_RCP45_Sub,70.0,164.0,6024.695374,10863.629792,0.0,5104.727645,1.0,11204.866489,22334.289509
1ft_Sub,111.0,247.0,9525.979890,16759.390087,2.0,8182.733181,1.0,17429.678973,35138.392044


of the 154 km of burried water electrical and power lines the model estimates that 70 km or 45% of these assets will be located in permanantly saturated soils under a SLR scenario of 2.4 m with a projected future climate following the RCP4.5 trajectroy

The model also predicts stark impacts to surface based infrastructure, whereas in the SLR of 2.4 m plus RCP 4.5 climate scenario 28% of present day roads, 13.6% of mapped buildings and one out of five current water booster stations (20%) will be inundated with standing water on the surface.   

### Some context and results notes

TO consider: 
- should report results of surface inundated features (DtW_level of 0) and subsurface inundated features (DtW_level of 1) too


The calibrated base scenario seems to do a reasonably good job of showing where average condition groundwater inundation occurrs. Areas in the present day base scenario where water reaches the ground surface match well with areas where there are canals or other persistant nearshore water features (Figure if need) 


Sewer lines includes all force and gravity mains on the asbuilts which we recognize to be an incomplete dataset, but is the best available provided by the utility. 

In present day conditions, of the 9.11 km total of water mains (44.1 km of distribution, 47.1 km of transmission), if we assume a 1 m depth then a total of 3.4 km are consistently inundated with water = 3.7%
In present day conditions, of the 42.3 km total of sewer lines, if we assume a 1 m depth then a total of 1.4 km are consistently inundated with water = 3.3%  
[STILL NEED BURIED ELECTRICAL LINES] 





In [9]:
123359.6/20


6167.9800000000005

In [11]:
(123359.6/20)/(24*3)

85.66638888888889